In [114]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [115]:
def driver_setup():# Assigning a Headless Firefox Driver
    options = webdriver.FirefoxOptions()
    options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
    options.add_argument('E:/Programs/geckodriver.exe')
    # options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Firefox(options=options)
    return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [116]:
url = r'file:///C:/Users/PM/Desktop/PDF%20parsing/gmat-for-dummies-7th-edition.pdf'
driver = driver_setup() # for each link we are setting a new Firefox instance
driver.get(url)
    

The version of firefox cannot be detected. Trying with latest driver version


In [117]:
pagenavigation = driver.find_element(By.CSS_SELECTOR, 'div.splitToolbarButton.hiddenSmallView').find_elements(By.CSS_SELECTOR, 'button[title="Previous Page"], button[title="Next Page"]')
print(len(pagenavigation))

2


In [118]:
pagenavigation[0].click()

In [119]:
pages = driver.find_element(By.ID, 'numPages').text
pages

'(1 of 411)'

In [120]:
com_text = []
for page in range(345,358):
    time.sleep(3)

    div = driver.find_element(By.CSS_SELECTOR, f'div.page[data-page-number="{page}"]')
    span = div.find_elements(By.TAG_NAME, "span")
    i=0
    while i < len(span):
        text = span[i].text
        com_text.append(text)
        i += 1
    pagenavigation[1].click()    

In [121]:
com_text

['CHAPTER 21',
 ' ',
 'GMAT Practice Test',
 ' ',
 '333',
 'Section 4: Verbal',
 'TIME:',
 ' ',
 '75 minutes for 41 questions',
 'DIRECTIONS:',
 ' ',
 'Follow these directions for each of the three question types:',
 '•',
 '',
 ' ',
 'Sentence-correction questions:',
 ' ',
 'Choose the answer choice that best phrases the underlined words',
 'according to the rules of standard English. The first answer choice duplicates the phrasing of the under',
 '-',
 'lined portion; the other four choices provide alternative phrasings. Choose the one that rephrases the',
 'sentence in the clearest, most grammatically correct manner.',
 '•',
 '',
 ' ',
 'Reading-comprehension questions:',
 ' ',
 'Choose the best answer to every question based on what the',
 'passage states directly or indirectly.',
 '•',
 '',
 ' ',
 'Critical-reasoning questions:',
 ' ',
 'Pick the answer choice that best answers the question about the argument',
 'provided.',
 'Questions 1–6 refer to the following passage, which',
 

In [122]:
question_list = []
choice_list = []
ans_list = []

start_saving_question = False
start_saving_choice = False

i = 0
c = 0 # CORRECT choice finder, Itirates back to find the choice.
while i < len(com_text):
    qnopat = r'(?<!\$)\b(?:[1-9]|[1-9][0-9]|[1-9][0-9][0-9])\.\B'
    # choipat = re.compile(r'\b(?:[A-E])\.\B')
    choipat = r'(?<!\S)\([A-M]\)(?!\S)'
    text = com_text[i]
    
    if re.search(qnopat, text) and ('450.' not in text):
    # if 'Stem' in text:
        start_saving_question = True
        start_saving_choice = False

    elif re.search(choipat, text):
        start_saving_choice = True
        start_saving_question = False
    
    elif 'options'.lower() in text.lower():
        start_saving_question = False

    elif ('correct'.lower() in text.lower()):
        start_saving_question = False
        start_saving_choice = False
    
    elif ('I.' in text) or ('II.' in text) or ('III.' in text) or ('Explanation' in text):
        start_saving_question = False
        start_saving_choice = False
        
    if start_saving_question:
        question_list.append(text)
        
    if start_saving_choice:
        if 'option' not in text:
            choice_list.append(text)

    if re.search(qnopat, text):
        if c==0:
            c+=1
        else:
            j = i-1
            sub = []
    i+=1
    

In [123]:
print(ans_list,'\n',question_list,'\n',choice_list)
print(len(ans_list),len(question_list),len(choice_list))

[] 
 ['1.', '', ' ', 'The main idea of the first paragraph is best', 'expressed as', '2.', '', ' ', 'Which of the following might the author of the', 'passage consider an “indirect loss” associated', 'with a disaster?', '3.', '', ' ', 'According to the passage, an important', '', 'distinction between hazards and risks is', '4.', '', ' ', 'The passage is primarily concerned with', '5.', '', ' ', 'Which of the following best describes the pur-', 'pose of the fourth paragraph in relation to the', 'passage as a whole?', '6.', '', ' ', 'Which of the following logically follows the', 'information given in the passage?', '7.', '', ' ', 'Before the primary school can open to the', 'public in time for the fall semester, the crew in', 'charge of the project', ' ', 'must finish construction,', 'cleaning, and safety-proofing it’s interior', '.', '8.', '', ' ', 'So many accounts of this', ' ', 'continents’ past', 'begin with Europeans striding ashore, claim-', 'ing this “newfound land” and its huma

In [124]:
qnopat = r'\b(?:[1-9]|[1-9][0-9]|[1-9][0-9][0-9])\.\B'

# Initialize variables to store question numbers and content
question_numbers = []
question_content = []

startconcat = False

question = []
for item in question_list:
    match = re.match(qnopat, item)
    if match and ('450.' not in item):
    # if 'Stem' in item:
        if startconcat:
            question_content.append(' '.join(question))
        question = []
        question_numbers.append(item)
        startconcat = True
        continue
    if startconcat:
        question.append(item)

if question and startconcat:
    question_content.append(' '.join(question))

#------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------

In [125]:
choiinpat = r'(?<!\S)\([A-M]\)(?!\S)'
# choipat = re.compile(r'\b(?:[A-E])\.\B')
# Initialize variables to store choices
choice_numbers = []
choice_subchoices = []

current_choice_number = None
current_subchoices = []

for item in choice_list:
    cmatch = re.match(choipat, item)
    if cmatch:
        if current_choice_number is not None:
            choice_numbers.append(current_choice_number)
            choice_subchoices.append(current_subchoices)
            current_subchoices = []
        
        current_choice_number = item
        continue
    
    if current_choice_number is not None:
        current_subchoices.append(item)

# Append the last set of choices
if current_choice_number is not None:
    choice_numbers.append(current_choice_number)
    choice_subchoices.append(current_subchoices)
    
choice_data = []
current_values = []

one_r = {}
for i in range(len(choice_numbers)):
    if choice_numbers[i] == 'A.':
        if one_r != {}:
            choice_data.append(one_r)
        one_r = {}
    one_r['choice_'+choice_numbers[i]] = ' '.join(choice_subchoices[i])

# Append the last one_r if it's not empty
if one_r:
    choice_data.append(one_r)

In [126]:
print(len(choice_numbers),'\n',len(choice_subchoices))

205 
 205


In [127]:
for i in range(len(choice_numbers)):
    print(choice_numbers[i],'-------',choice_subchoices[i])

(A) ------- ['', ' ', 'The factors that make an area desirable', 'are also those that can pose the most risk.']
(B) ------- ['', ' ', 'The Hawaiian Islands would not exist', 'if not for powerful and explosive', 'volcanoes.']
(C) ------- ['', ' ', 'Floods, volcanoes, and earthquakes pose', 'threats to the natural environment.']
(D) ------- ['', ' ', 'Humans must learn to live with natural', 'hazards such as volcanoes and tsunamis.']
(E) ------- ['', ' ', 'Natural hazards are most prevalent in', 'areas that are sunny and warm.']
(A) ------- ['', ' ', 'Desecration of a library due to vandalism']
(B) ------- ['', ' ', 'Damage to a school building in a fire']
(C) ------- ['', ' ', 'Loss of retail clothing sales due to a mall', 'flood']
(D) ------- ['', ' ', 'Death of a ranch’s livestock due to vol-', 'canic eruption']
(E) ------- ['', ' ', 'Destruction of a pavilion due to a', 'hurricane']
(A) ------- ['', ' ', 'risks occur naturally, while hazards arise', 'because of human actions']
(B) --

In [ ]:
# for i in range(160): #-------------------------------------------choice correctin
#     norem = []
#     norem.append(choice_numbers[i][2:])
#     choice_numbers[i] = choice_numbers[i][:2]
#     norem.extend(choice_subchoices[i])
#     choice_subchoices[i] = norem
#     print(choice_numbers[i],'-------',choice_subchoices[i])

In [128]:
choice_data = []
current_values = []

one_r = {}
for i in range(len(choice_numbers)):
    if choice_numbers[i] == '(A)':
        if one_r != {}:
            choice_data.append(one_r)
        one_r = {}
    one_r['choice_'+choice_numbers[i]] = ' '.join(choice_subchoices[i])

# Append the last one_r if it's not empty
if one_r:
    choice_data.append(one_r)

In [271]:
qno = []
qco = []
qs1no = []
qs1co = []
qs2no = []
qs2co = []
for i in range(len(question_numbers)):
    if i%3==0:
        qno.append(question_numbers[i])
        qco.append(question_content[i])
    if i%3==1:
        qs1no.append(question_numbers[i])
        qs1co.append(question_content[i])
    if i%3==2:
        qs2no.append(question_numbers[i])
        qs2co.append(question_content[i])
print(len(qno),len(qco),len(qs1no),len(qs1co),len(qs2no),len(qs2co))

10 10 10 10 10 10


In [ ]:

for i in range(len(question_numbers)):

In [129]:
for i in range(len(question_numbers)):
    # question_numbers[i] = q 
    print(question_numbers[i],'------',question_content[i])
    # q+=1

# q = 1
# for i in range(len(question_numbers)):
#     question_numbers[i] = q 
#     print(question_numbers[i],'------',question_content[i])
#     q+=1

# for i in range(len(qno)):
#     print(qno[i],'------',qco[i])

1. ------    The main idea of the first paragraph is best expressed as
2. ------    Which of the following might the author of the passage consider an “indirect loss” associated with a disaster?
3. ------    According to the passage, an important  distinction between hazards and risks is
4. ------    The passage is primarily concerned with
5. ------    Which of the following best describes the pur- pose of the fourth paragraph in relation to the passage as a whole?
6. ------    Which of the following logically follows the information given in the passage?
7. ------    Before the primary school can open to the public in time for the fall semester, the crew in charge of the project   must finish construction, cleaning, and safety-proofing it’s interior .
8. ------    So many accounts of this   continents’ past begin with Europeans striding ashore, claim- ing this “newfound land” and its human inhabitants for its respective empires.
9. ------    Injera, an East African flatbread, has been

In [87]:
sudf = pd.DataFrame() #---------------------------------------------------------sub questions aligner

data = {}
therelist = []
subq = False
for i in range(len(question_numbers)):
    if (question_numbers[i] not in therelist) and (subq == False):
        
        therelist.append(question_numbers[i])
        
        if data != {}:
            tempdf = pd.DataFrame(data,index=[0])
            sudf = pd.concat([sudf, tempdf], ignore_index=True)
        
        data = {}
        
        data['qno'] = question_numbers[i]
        data['qco'] = question_content[i]
        
    else:
        subq = True
        if question_numbers[i] == '1.':
            data['qnosu1'] = question_content[i]
        elif question_numbers[i] == '2.':
            data['qnosu2'] = question_content[i]
            subq = False    

tempdf = pd.DataFrame(data, index=[0])
sudf = pd.concat([sudf, tempdf], ignore_index=True)
sudf

,qno,qco,qnosu1,qnosu2
0,1.,If a doughnut-making machine is operated fo...,The machine produces doughnuts at a rate of...,The doughnut company sells cases of doughnu...
1,2.,The figure depicts a rectangle and a circle...,y 6,The area of the rectangle is 18.
2,3.,10 15 2 3 3 1 xy x y y If x...,NaN,NaN
3,4.,A stand at a farmer’s market is selling pea...,NaN,NaN
4,5.,"If y 4 , what is the value of 2 7...",x y 3 2,2 5 0 x y
5,6.,The table shows the peak price each year fo...,NaN,NaN
6,7.,How many kilometers does a bullet train tra...,It takes the bullet train two hours to get ...,The bullet train travels at an average spee...
7,8.,"If y 1 , what is the value of the foll...",NaN,NaN
8,9.,"When x 0 , 2 1 3 2 x x can be...",NaN,NaN
9,10.,Sophia is buying a smartphone and a phone c...,The sale that the store is offering is 15 p...,The original price of the smartphone alone ...


In [ ]:
therelist

In [132]:
choice_df = pd.DataFrame(choice_data)
# question_data = {'Question Number': qno, 'Question Content': qco, 'SubQuestion1':qs1co,'SubQuestion2':qs2co}
question_data = {'Question Number': question_numbers, 'Question Content': question_content}
df = pd.concat([pd.DataFrame(question_data), choice_df], axis=1)
# df = pd.concat([sudf, choice_df], axis=1)
# tdf = df.copy()

In [133]:
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E)
0,1.,The main idea of the first paragraph is bes...,The factors that make an area desirable are...,The Hawaiian Islands would not exist if not...,"Floods, volcanoes, and earthquakes pose thr...",Humans must learn to live with natural haza...,Natural hazards are most prevalent in areas...
1,2.,Which of the following might the author of ...,Desecration of a library due to vandalism,Damage to a school building in a fire,Loss of retail clothing sales due to a mall...,Death of a ranch’s livestock due to vol- ca...,Destruction of a pavilion due to a hurricane
2,3.,"According to the passage, an important dis...","risks occur naturally, while hazards arise ...","hazards result from risks, and risks result...","hazards can lead to disasters, while risks ...","hazards are not under human control, while ...",risks are harder to quantify than hazards
3,4.,The passage is primarily concerned with,describing the causes and impacts of natura...,assessing the impact that disasters render...,depicting the various ways human beings may...,raising awareness of the loss of human live...,explaining that natural disasters are not u...
4,5.,Which of the following best describes the p...,It uses numerical data and metrics to descr...,It emphasizes how little is actually known ...,It outlines the differences between hazards...,It provides sensory details about spe- cifi...,It applies statistical data to emphasize th...
5,6.,Which of the following logically follows th...,The number of unreported deaths in the 2010...,"In the years 2010 and 2004, there were more...",The number of deaths due to natural disaste...,Economic costs are more frequently unreport...,"Areas of high hazard, such as Japan’s Tohok..."
6,7.,Before the primary school can open to the p...,"must finish construction, cleaning, and saf...","must finish construction of, cleaning, and ...","must finish constructing, cleaning and safe...","must finish constructing, cleaning, and saf...","must finish construction, cleaning and safe..."
7,8.,So many accounts of this continents’ past...,continents’ past begin with Europeans strid...,continent’s past begin with Europeans strid...,continents’ past begin with Europeans strid...,continent’s past begins with Europeans stri...,continent’s past begins with Europeans stri...
8,9.,"Injera, an East African flatbread, has been...","are still used by many Ethiopians today, wh...","are still used by many Ethiopians today, wh...","it is still used today by many Ethiopians, ...",is still used today by many Ethiopians to f...,"is still used by many Ethiopians today, who..."
9,10.,The size of oceanic waves is a function of ...,The degree of erosion to which a coast- lin...,The size of oceanic waves will not fluc - t...,The fetch of winds is related to the shape ...,The size of oceanic waves is related to the...,The average velocity of the wind in an area...


In [274]:
for index, row in df.iterrows():#-----------------------------------------------------------question
    question_number = str(row['Question Number'])
    if not question_number[:-1].isdigit():
        # Extract the text from the next column
        nearby_text = row['Question Content']
        if isinstance(nearby_text, str):
            # Append the text to the nearby question content
            updated_content = question_number+' '+nearby_text
            df.at[index, 'Question Content'] = updated_content
df

,Question Number,Question Content,SubQuestion1,SubQuestion2,choice_A.,choice_B.,choice_C.,choice_D.,choice_E.
0,121.,A teacher distributed some pens and some p...,The teacher distributed 180 pens and 40 penc...,The total number of items received by each s...,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
1,122.,"The number 2135 , when divided by a number ...","The number 2315, when divided by 𝑛 , also ...","The number 2585, when divided by 𝑛 , also ...",Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
2,123. What is the remainder when,123. What is the remainder when 2 𝑛 is div...,"𝑛 , when divided by 6, leaves remainder 5.","3 𝑛 , when divided by 6, leaves remainder 3....",Stat ement (1) alone is sufficient but state...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is s ufficient,Statements (1) and (2) together are not suff...
3,124.,"The remainder , when a number 𝑛 is div...",𝑛 is a positive number having 8 as a factor.,𝑛 is a positive number having 6 as a facto...,Statement (1) alone is sufficient but statem...,St atement (2) alone is sufficient but state...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
4,125.,Is 𝑛 12 an integer?,𝑛 2 144 is an integer.,𝑛 6 is an integer. is divisible by 144. Ho...,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
5,126. If,"126. If 𝑥 and 𝑦 are integers, w hat is...",𝑥 𝑦 = 64 .,𝑦 𝑥 = 36 .,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, b...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
6,127.,127 Can 𝑛 192 be an integer?,𝑛 is a multiple of 24 but not 16.,𝑛 is a multiple of 16 but not 24. and also...,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are suffici ent, bu...",,Statements (1) and (2) together are not suff...
7,128.,If 𝑎 and 𝑏 are single digits from 0 ...,The number 25 𝑎 7 𝑏 is divisible by 36.,The number 𝑎 1 𝑏 is divisible by 4. is d...,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...
8,129.,What is the remainder obtained when 𝑛 is...,( 2 𝑛 + 2 ) is divisible by 14.,( 𝑛 2 − 1 ) is divisible by 7. is divi...,Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are no t suf...
9,130.,𝑝 is the value of the integer such that ...,𝑝 is the least such integer.,"3 𝑝 + 2 < 500 . is 5. Thus, the rema...",Statement (1) alone is sufficient but statem...,Statement (2) alone is sufficient but statem...,"Both statements together are sufficient, but...",Each statement alone is sufficient,Statements (1) and (2) together are not suff...


In [50]:
df['Question Number'] = df['Question Number'].apply(lambda x: (x[:3] if len(x) > 4 else x))#-----------------------------------------------------------question
df['Question Content'] = df.apply(lambda row: re.sub(r'^\d+\.\s*', '', row['Question Content']) if pd.notnull(row['Question Content']) else '', axis=1)
# correct numbers to have numbers and questions to have question

TypeError: object of type 'int' has no len()

In [ ]:
df

In [104]:
# new_column_names = {'qno':'Question Number', 'qco':'Question Content', 'qnosu1':'SubQuestion1', 'qnosu2':'SubQuestion2'}
# df = df.rename(columns=new_column_names)

In [134]:
df['Question Number'] = df['Question Number'].str.replace('.','')
df['Question Number'] = df['Question Number'].astype('int')
df = df.sort_values(by='Question Number')
df = df.reset_index(drop=True)

C:\Users\PM\AppData\Local\Temp\ipykernel_14552\2984988848.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Question Number'] = df['Question Number'].str.replace('.','')


In [135]:
df.columns

Index(['Question Number', 'Question Content', 'choice_(A)', 'choice_(B)',
       'choice_(C)', 'choice_(D)', 'choice_(E)'],
      dtype='object')

In [136]:
df

,Question Number,Question Content,choice_(A),choice_(B),choice_(C),choice_(D),choice_(E)
0,1,The main idea of the first paragraph is bes...,The factors that make an area desirable are...,The Hawaiian Islands would not exist if not...,"Floods, volcanoes, and earthquakes pose thr...",Humans must learn to live with natural haza...,Natural hazards are most prevalent in areas...
1,2,Which of the following might the author of ...,Desecration of a library due to vandalism,Damage to a school building in a fire,Loss of retail clothing sales due to a mall...,Death of a ranch’s livestock due to vol- ca...,Destruction of a pavilion due to a hurricane
2,3,"According to the passage, an important dis...","risks occur naturally, while hazards arise ...","hazards result from risks, and risks result...","hazards can lead to disasters, while risks ...","hazards are not under human control, while ...",risks are harder to quantify than hazards
3,4,The passage is primarily concerned with,describing the causes and impacts of natura...,assessing the impact that disasters render...,depicting the various ways human beings may...,raising awareness of the loss of human live...,explaining that natural disasters are not u...
4,5,Which of the following best describes the p...,It uses numerical data and metrics to descr...,It emphasizes how little is actually known ...,It outlines the differences between hazards...,It provides sensory details about spe- cifi...,It applies statistical data to emphasize th...
5,6,Which of the following logically follows th...,The number of unreported deaths in the 2010...,"In the years 2010 and 2004, there were more...",The number of deaths due to natural disaste...,Economic costs are more frequently unreport...,"Areas of high hazard, such as Japan’s Tohok..."
6,7,Before the primary school can open to the p...,"must finish construction, cleaning, and saf...","must finish construction of, cleaning, and ...","must finish constructing, cleaning and safe...","must finish constructing, cleaning, and saf...","must finish construction, cleaning and safe..."
7,8,So many accounts of this continents’ past...,continents’ past begin with Europeans strid...,continent’s past begin with Europeans strid...,continents’ past begin with Europeans strid...,continent’s past begins with Europeans stri...,continent’s past begins with Europeans stri...
8,9,"Injera, an East African flatbread, has been...","are still used by many Ethiopians today, wh...","are still used by many Ethiopians today, wh...","it is still used today by many Ethiopians, ...",is still used today by many Ethiopians to f...,"is still used by many Ethiopians today, who..."
9,10,The size of oceanic waves is a function of ...,The degree of erosion to which a coast- lin...,The size of oceanic waves will not fluc - t...,The fetch of winds is related to the shape ...,The size of oceanic waves is related to the...,The average velocity of the wind in an area...


In [137]:
cust = ['passage_content','Question Number', 'Question Content', 'choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)', 'ans']
# cust = ['Question Number', 'Question Content', 'SubQuestion1', 'SubQuestion2','choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)']
# cust = ['Question Number', 'Question Content', 'choice_(A)', 'choice_(B)', 'choice_(C)', 'choice_(D)', 'choice_(E)']
# cust = ['Question Number', 'Question Content', 'choice_A.', 'choice_B.', 'choice_C.', 'choice_D.', 'choice_E.']
# cust = ['Question Number', 'Question Content', 'SubQuestion1', 'SubQuestion2', 'choice_A.', 'choice_B.','choice_C.', 'choice_D.', 'choice_E.']

df = df.reindex(columns = cust)

In [138]:
# df['passage']= ' '.join(passage_list)
df.to_excel('Gmat_practiece_verbal.xlsx', index = False)